[Reference](https://mahdijafaridev.medium.com/apis-that-scale-lessons-i-learned-building-with-fastapi-cad20ad723da)

# 1. Use Async for IO-Bound Workloads, Not CPU Tasks

In [9]:
from fastapi import FastAPI
import httpx

app = FastAPI()

@app.get("/async-task")
async def async_task():
    async with httpx.AsyncClient() as client:
        response = await client.get("https://httpbin.org/delay/1")
    return {"status": response.status_code}

# 2. Modularize Your API and Version Endpoints


In [10]:
from fastapi import APIRouter, FastAPI

app = FastAPI()

v1_router = APIRouter(prefix="/api/v1")
v2_router = APIRouter(prefix="/api/v2")

@v1_router.get("/items")
async def get_items_v1():
    return [{"id": 1, "name": "Legacy Item"}]

@v2_router.get("/items")
async def get_items_v2():
    return [{"id": 1, "name": "Item v2", "description": "More details"}]

app.include_router(v1_router)
app.include_router(v2_router)

# 3. Optimize Database Access

In [11]:
!pip install sqlmodel

In [13]:
from sqlmodel import SQLModel, Field, create_async_engine, select
from sqlmodel.ext.asyncio.session import AsyncSession
from fastapi import FastAPI, Depends

app = FastAPI()
engine = create_async_engine("sqlite+aiosqlite:///test.db")

class Item(SQLModel, table=True):
    id: int = Field(default=None, primary_key=True)
    name: str

async def get_session():
    async with AsyncSession(engine) as session:
        yield session

@app.get("/items")
async def read_items(session: AsyncSession = Depends(get_session)):
    result = await session.execute(select(Item))
    return result.scalars().all()

# 4. Employ Caching Wisely

In [14]:
from fastapi import FastAPI
from fastapi_cache import FastAPICache
from fastapi_cache.backends.inmemory import InMemoryBackend
from fastapi_cache.decorator import cache
import asyncio

app = FastAPI()

@app.on_event("startup")
async def startup():
    FastAPICache.init(InMemoryBackend())

@app.get("/cached-items")
@cache(expire=60)
async def get_cached_items():
    await asyncio.sleep(1)  # simulate expensive work
    return {"items": ["item1", "item2"]}

# 5. Monitor, Log, and Profile Continuously

In [15]:
import sentry_sdk
from fastapi import FastAPI

sentry_sdk.init(dsn="YOUR_SENTRY_DSN")
app = FastAPI()

# 6. Automate Testing and CI/CD

In [16]:
from fastapi.testclient import TestClient
from main import app

client = TestClient(app)

def test_items():
    response = client.get("/items")
    assert response.status_code == 200
    assert isinstance(response.json(), list)

# 7. Manage Configurations and Secrets Securely


In [20]:
# pip install pydantic-settings
from pydantic_settings import BaseSettings


class Settings(BaseSettings):
    db_url: str
    secret_key: str

    class Config:
        env_file = ".env"

settings = Settings()